In [1]:
!pip3 install anthropic python-dotenv --quiet

You should consider upgrading via the '/Users/elie/.pyenv/versions/3.10.4/bin/python3.10 -m pip install --upgrade pip' command.


In [2]:
import anthropic 
import dotenv

dotenv.load_dotenv()

False

In [38]:
INSTRUCTORS = {
    "alex": {
        "first_name": "Alex",
        "email": "elie@anthropic.com",
    },
    "colt": {
        "name": "Colt",
        "email": "colt@anthropic.com",
    },
    "maggie": {
        "name": "Maggie",
        "email": "maggie@anthropic.com",
    },
}

def find_instructor_email(username):
    return INSTRUCTORS.get(username, {}).get("email", "Instructor not found")


In [37]:
import anthropic

client = anthropic.Client()

tools = [
    {
        "name": "find_instructor_email",
        "description": "Get an instructors email",
        "input_schema": {
            "type": "object",
            "properties": {
                "username": {
                    "type": "string",
                    "description": "The username of the instructor"
                },
            },
            "required": ["username"]
        }
    }
]

response = client.messages.create(
    model="claude-3-opus-20240229",
    tools=tools,
    max_tokens=1024,
    messages=[{'role': 'user',
               'content': 'What is the instructor Colt\'s email? His username is colt.'}]
)

if response.stop_reason == "tool_use":
    # a slightly more defensive way to get the response content instead of using -1

    tool_use = tool_use = next(
        block for block in response.content if block.type == "tool_use")

    instructor = find_instructor_email(tool_use.input['username'])



In [42]:
response.content[-1]

ToolUseBlock(id='toolu_01YX82eYXRQrcvvRutt1ioMW', input={'username': 'elie'}, name='find_instructor_email', type='tool_use')

In [ ]:
final_result = client.messages.create(
    model = "claude-3-opus-20240229",
    tools = tools,
    max_tokens = 1024,
    messages = [
        {
            'role': 'user',
            'content': 'What is the instructor Colt\'s email? His username is colt.'
        },
        { "role": "assistant", "content": response.content },
        {
            'role': 'user',
            "content": [
                {
                    "type": "tool_result",
                    "tool_use_id": response.content[-1].id,
                    "content": instructor,
                }
            ]
        }
    ]
)



In [7]:
print(final_result.content[0].text)

Elie's email address is elie@anthropic.com.
